In [4]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

In [1]:
df = pd.read_csv('clean_df.csv')
df.head()

,Unnamed: 0,RESP_ID,Collector ID,Start Date,End Date,IP Address,Email Address,First Name,Last Name,Custom Data 1,...,i1_1a,i1_1b_values,i1_1b,i1_1c_values,i1_1c,i4_1,i5_1_values,i5_1,i6_1_values,i6_1
0,0,12477972045,263393441,2021-03-09 22:21:38,2021-03-09 23:31:00,163.116.193.114,NaN,NaN,NaN,NaN,...,1.0,4.0,1.0,Agree,1.0,1.0,4.0,1.0,NaN,NaN
1,1,12477891999,263393441,2021-03-09 22:20:17,2021-03-09 22:30:11,163.116.193.117,NaN,NaN,NaN,NaN,...,1.0,4.0,1.0,Agree,1.0,0.0,4.0,1.0,MI/MI2 support had a large positive effect on ...,1.0
2,2,12477891089,263393441,2021-03-09 22:19:45,2021-03-09 23:09:10,163.116.193.117,NaN,NaN,NaN,NaN,...,1.0,3.0,0.0,Agree,1.0,0.0,3.0,0.0,NaN,NaN
3,3,12477471535,263393441,2021-03-09 19:03:06,2021-03-09 21:11:28,101.128.127.169,NaN,NaN,NaN,NaN,...,1.0,4.0,1.0,Agree,1.0,1.0,4.0,1.0,MI/MI2 support had a moderate positive effect ...,1.0
4,4,12477102564,263393441,2021-03-09 16:47:52,2021-03-09 17:02:34,72.234.176.36,NaN,NaN,NaN,NaN,...,1.0,4.0,1.0,Agree,1.0,1.0,4.0,1.0,MI/MI2 support had little effect on the qualit...,0.0


In [2]:
profile = ProfileReport(df, title="Pandas Profiling Report",explorative=True)

In [3]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/86 [00:00<?, ?it/s]

C:\Users\Erik\Anaconda3\envs\dev-env\lib\site-packages\pandas_profiling\model\correlations.py:101: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
profile.to_file("mi2-survey-variable-report.html")

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Human interpretable
This is a human interpretable report with just the questions


In [5]:
dff = pd.read_excel(r'C:\Users\Erik\OneDrive - Environmental Incentives/Downloads/mi2-data.xlsx', 'RAW_INPUT', header=[0,1], engine='openpyxl', date_parser=[3,4])

# THIS CAN ALL BE DONE WITH ONLY THE LEVEL 1
# (1) strip columns of returns
# (2) update instances of 'previous response' to point to previous column
columns = []
for idx, column in enumerate(dff.columns):
    level1 = str(column[0]).strip()
    level2 = str(column[1]).strip()
    if 'previous response' in level1.lower():
        previous_question = dff.columns[idx-1][0]
        new_level1 = level1 + '(' + previous_question + ')'
        columns.append((new_level1, level2))
    else:
        columns.append((level1, level2))
dff.columns = pd.MultiIndex.from_tuples(columns, names=["first", "second"])

# concatenate identical questions and drop response row
level_ones = dff.columns.get_level_values('first')
column_filters = [level_ones.get_loc(c) for c in level_ones]  # -> int if unique else boolean array 
series = []
already_seen = set()
for i in column_filters:
    if type(i)==int:
        s = dff.iloc[:,i].copy()
        s.name = s.name[0]
        series.append(s)
    elif type(i)==np.ndarray:
        s = dff.iloc[:,i].copy()
        name = s.columns[0][0]
        if name not in already_seen:
            s = s.droplevel('second', axis=1)
            s = s.apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)
            s.name = name
            series.append(s)
            already_seen.add(name)
human_df = pd.concat(series, axis=1)

# convert all empty cells to nan
human_df = human_df.replace(r'^\s*$', np.nan, regex=True)

# save to csv
human_df.to_csv('human_df.csv')

In [7]:
interpretable_profile = ProfileReport(human_df, title="Pandas Profiling Report", minimal=True)

In [9]:
interpretable_profile.to_file("mi2-survey-question-report.html")

Summarize dataset:   0%|          | 0/67 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]